In [1]:
##Import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta

# import all libraries and dependencies for data visualization
pd.options.display.float_format='{:.4f}'.format
plt.rcParams['figure.figsize'] = [8,8]
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_colwidth', -1) 
pd.set_option("display.max_rows", 500)
sns.set(style='darkgrid')
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker


# import all libraries and dependencies for machine learning
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler


from sklearn.compose import make_column_transformer


import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

from scipy import stats
from scipy.stats import norm, kurtosis, skew

# Import specific libraries
import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.set_option('expand_frame_repr', False)



In [2]:
##FILTRAR: mas_cols  = all_data.filter(like='Mas', axis=1).columns

In [3]:
#Loading updated dataset

%store -r all_data
%store -r y_train
all_data.shape, y_train.shape

((2905, 225), (1446,))

In [4]:
###Correlation data for analysis
data = all_data.iloc[:len(y_train), :]
data["SalePrice"] = y_train

#Correlation dataframe
df = data.corr()


In [5]:
##MODULO: GENERA DATAFRAME CON LOS FEATURES MAS CORRELACIONADOS RECOMENDANDO LOS DROPABLES
#INPUT: df = cuadro de correlaciones (data,corr())
#OUTPUT: sum_final, summary de features mas correlacionados, sus correlaciones con target y lista conrecomendacion de eliminar el
#####de menor correlacion con target (Dropables)
#Nota: se filtran los features con correlacion con target > 0.26

def feat_corr(df, target):
    for i in df.index:
        cols = list(set(df.index) - set([i]))
        maximo = np.abs(df.loc[i, cols]).max()
        for c in cols:
            name_max = "None"
            if np.abs(df.loc[i, c]) == maximo:
                name_max = c
                df.loc[i, "Feature_w_max_corr"] = c
                df.loc[i, "Max_correlation"] = df.loc[i, c] 
                df.loc[i, "F1_corr_Target"] = df.loc[i, target]
                df.loc[i, "F2_corr_Target"] = df.loc[c, target]
                if np.abs(df.loc[i, "F1_corr_Target"]) > np.abs(df.loc[i, "F2_corr_Target"]):
                    df.loc[i, "Dropable"] = c
                elif  np.abs(df.loc[i, "F1_corr_Target"]) < np.abs(df.loc[i, "F2_corr_Target"]):
                    df.loc[i, "Dropable"] = i
                else:
                    df.loc[i, "Dropable"] = i
                    
    summary =  df[["Feature_w_max_corr", "Max_correlation", "F1_corr_Target", "F2_corr_Target", "Dropable"]].sort_values(by = "F1_corr_Target", ascending = False)
#     sum_final = summary[(abs(summary["F1_corr_Target"]) > 0.26) & (abs(summary["F2_corr_Target"]) > 0.26)]
    sum_final = summary[(abs(summary["Max_correlation"]) > 0.9) ]

    dropables = list(sum_final["Dropable"].unique())
    return sum_final, dropables

sum, dropables = feat_corr(df, "SalePrice")

sum


,Feature_w_max_corr,Max_correlation,F1_corr_Target,F2_corr_Target,Dropable
TotalPoints,OverallQual,0.9705,0.8513,0.7980,OverallQual
OverallQual,TotalPoints,0.9705,0.7980,0.8513,OverallQual
GarageArea_x_Car,GarageArea,0.9343,0.7127,0.6491,GarageArea
TotalBsmtSF_x_Bsm,TotalBsmtSF,0.9424,0.6814,0.6489,TotalBsmtSF
GarageCars,GarageArea_x_Car,0.9087,0.6525,0.7127,GarageCars
GarageArea,GarageArea_x_Car,0.9343,0.6491,0.7127,GarageArea
TotalBsmtSF,TotalBsmtSF_x_Bsm,0.9424,0.6489,0.6814,TotalBsmtSF
BsmtSFPoints,BsmtQual,0.9218,0.6432,0.5855,BsmtQual
1stFlrSF,TotalBsmtSF_x_Bsm,0.9057,0.6292,0.6814,1stFlrSF
BsmtQual,BsmtSFPoints,0.9218,0.5855,0.6432,BsmtQual


In [6]:
dropables

['OverallQual',
 'GarageArea',
 'TotalBsmtSF',
 'GarageCars',
 'BsmtQual',
 '1stFlrSF',
 'SaleCondition_Partial',
 'Exterior1st_VinylSd',
 'GarageCond',
 'RoofStyle_Gable',
 'Exterior2nd_CmentBd',
 'PoolArea',
 'MiscFeature_NA',
 'New',
 'Remod',
 'Exterior1st_CBlock',
 'Exterior2nd_CBlock',
 'MSSubClass_80',
 'MSSubClass_45',
 'MSSubClass_190',
 'MSSubClass_90',
 'BldgType_Duplex',
 'Exterior2nd_MetalSd',
 'GarageNewest',
 'GarageType_NA',
 'YearRemodAdd']

In [7]:
viejo = list(['FireplaceQu', 'BsmtSFPoints', 'TotalBsmtSF', 'GarageArea', 'GarageCars', 'OverallQual', 'GrLivArea', 
               'TotalBsmtSF_x_Bsm', '1stFlrSF', 'PoolArea', 'LotArea', 'SaleCondition_Partial', 'Exterior1st_VinylSd',
               'GarageCond', 'HouseStyle_2Story', 'BsmtSFMultPoints', 'ScreenPorch', 'LowQualFinSF', 'BsmtFinSF2',
               '3SsnPorch'])
# sum.loc[viejo, :]

In [8]:
#Remove ConstructArea from dropables  because it is highloy correlated only with SalePrice
# dropables.remove("ConstructArea")


In [9]:
dropables

['OverallQual',
 'GarageArea',
 'TotalBsmtSF',
 'GarageCars',
 'BsmtQual',
 '1stFlrSF',
 'SaleCondition_Partial',
 'Exterior1st_VinylSd',
 'GarageCond',
 'RoofStyle_Gable',
 'Exterior2nd_CmentBd',
 'PoolArea',
 'MiscFeature_NA',
 'New',
 'Remod',
 'Exterior1st_CBlock',
 'Exterior2nd_CBlock',
 'MSSubClass_80',
 'MSSubClass_45',
 'MSSubClass_190',
 'MSSubClass_90',
 'BldgType_Duplex',
 'Exterior2nd_MetalSd',
 'GarageNewest',
 'GarageType_NA',
 'YearRemodAdd']

In [10]:
#3We drop the features in dropables from all_data
all_data.drop(columns = dropables, axis = 1, inplace = True)
# all_data.drop(columns = viejo, axis = 1, inplace = True)





In [11]:
all_data.shape

(2905, 199)

In [12]:
data_0 = all_data.copy()
%store data_0

Stored 'data_0' (DataFrame)


In [ ]:
viejo = ['FireplaceQu', 'BsmtSFPoints', 'TotalBsmtSF', 'GarageArea', 'GarageCars', 'OverallQual', 'GrLivArea', 
               'TotalBsmtSF_x_Bsm', '1stFlrSF', 'PoolArea', 'LotArea', 'SaleCondition_Partial', 'Exterior1st_VinylSd',
               'GarageCond', 'HouseStyle_2Story', 'BsmtSFMultPoints', 'ScreenPorch', 'LowQualFinSF', 'BsmtFinSF2',
               'TSsnPorch']

In [ ]:
####ANALISIS DE VIF 
##Anbalizaremos el train set completo, eliminando las features con alto VIF y luego el test set será
#actualizado en sus columnas para que coindida con lo que obtengamos en esta etapa

In [ ]:
y_train.shape

In [ ]:
#Creamos un nuevo x_train con las columnas seleccionadas. Ese sera el que se usara para modelar y del que se extraeran las variables recursivamente
train = all_data.iloc[:len(y_train), :]
x_train_rfe = train
x_train_rfe.shape

In [ ]:
#Evaluamos en base a p-value y VIF

#Adding a constant and building the first fitted model
import statsmodels.api as sm
x_train_rfec = sm.add_constant(x_train_rfe)
lm_rfe = sm.OLS(y_train, x_train_rfec).fit()


In [ ]:
#Generate a summary of the fitted model...
#print(lm_rfe.summary())

In [ ]:
###Generate the VIF para esta vuelta
series_1 = pd.Series([variance_inflation_factor(x_train_rfe.values, i) for i in range(x_train_rfe.shape[1])], index = x_train_rfe.columns) #se usa el xtrain sin la constante
series_1 = series_1.sort_values(ascending = False)
series_1

In [ ]:
##Eliminamos los features con VIF inf UNO POR CATEGORIA (exterior solo 1)
x_train_rfe_1 = x_train_rfe.drop(columns = ["Heating_GasA" ]  , axis = 1)

In [ ]:
#Repetimos el procedimiento...
#Add constant and fit the model
x_train_rfec_1 = sm.add_constant(x_train_rfe_1)
lm_rfe_1 = sm.OLS(y_train, x_train_rfec_1).fit()

In [ ]:
#Summary of the model
#print(lm_rfe_1.summary())

In [ ]:
#VIF
series_2 = pd.Series([variance_inflation_factor(x_train_rfe_1.values, i) for i in range(x_train_rfe_1.shape[1])], index = x_train_rfe_1.columns) #se usa el xtrain sin la constante
series_2 = series_2.sort_values(ascending = False)
series_2

In [ ]:
##Send file for testing so far
data_1 = all_data[x_train_rfe_1.columns]
%store data_1

In [ ]:
#Elimino todos los features en col_h UNO POR CATEGORIA
x_train_rfe_2 = x_train_rfe_1.drop(columns = ["Street"], axis = 1)

In [ ]:
#Repetimos el procedimiento...

#Add constant and fit the model
x_train_rfec_2 = sm.add_constant(x_train_rfe_2)
lm_rfe_2 = sm.OLS(y_train, x_train_rfec_2).fit()

#Summary of the model
#print(lm_rfe_2.summary())

In [ ]:
#VIF 2
series_3 = pd.Series([variance_inflation_factor(x_train_rfe_2.values, i) for i in range(x_train_rfe_2.shape[1])], index = x_train_rfe_2.columns) #se usa el xtrain sin la constante
series_3 = series_3.sort_values(ascending = False)
series_3


In [ ]:
##Send file for testing so far
data_2 = all_data[x_train_rfe_2.columns]
%store data_2

In [ ]:
#Elimino todos los features en col_h UNO POR CATEGORIA
x_train_rfe_3 = x_train_rfe_2.drop(columns = [ "TotalExtraPoints","PoolQC","FireplaceQu", "HeatingQC", "KitchenQual"], axis = 1)

In [ ]:
#Repetimos el procedimiento...

#Add constant and fit the model
x_train_rfec_3 = sm.add_constant(x_train_rfe_3)
lm_rfe_3 = sm.OLS(y_train, x_train_rfec_3).fit()

#Summary of the model
#print(lm_rfe_3.summary())

In [ ]:
#VIF 3
series_4 = pd.Series([variance_inflation_factor(x_train_rfe_3.values, i) for i in range(x_train_rfe_3.shape[1])], index = x_train_rfe_3.columns) #se usa el xtrain sin la constante
series_4 = series_4.sort_values(ascending = False)
series_4

In [ ]:
##Send file for testing so far
data_3 = all_data[x_train_rfe_3.columns]
%store data_3

In [ ]:
#Elimino FullBath por alto valor 
x_train_rfe_4 = x_train_rfe_3.drop(columns =["ConstructArea", "GrLivArea", "LandSlope", "BsmtUnfSF"], axis = 1)

#Repetimos el procedimiento...

#Add constant and fit the model
x_train_rfec_4 = sm.add_constant(x_train_rfe_4)
lm_rfe_4 = sm.OLS(y_train, x_train_rfec_4).fit()

#Summary of the model
#print(lm_rfe_4.summary())

In [ ]:
#VIF 4

series_5 = pd.Series([variance_inflation_factor(x_train_rfe_4.values, i) for i in range(x_train_rfe_4.shape[1])], index = x_train_rfe_4.columns) #se usa el xtrain sin la constante
series_5 = series_5.sort_values(ascending = False)
series_5

In [ ]:
##Send file for testing so far
data_4 = all_data[x_train_rfe_4.columns]
%store data_4

In [ ]:
#Elimino otro alto VIF
x_train_rfe_5 = x_train_rfe_4.drop(columns = ["Condition2_Norm", "BsmtSFPoints", 'FullBath', 'MSZoning_RL', 'Functional',"BsmtSFMultPoints",'ExterQual', 'GarageQual', 'TotalPoints', 'ExterCond','GarageType_Attchd',"BsmtCond"], axis = 1)

#Repetimos el procedimiento...

#Add constant and fit the model
x_train_rfec_5 = sm.add_constant(x_train_rfe_5)
lm_rfe_5 = sm.OLS(y_train, x_train_rfec_5).fit()

#Summary of the model
#print(lm_rfe_5.summary())



In [ ]:
#VIF 5

series_6 = pd.Series([variance_inflation_factor(x_train_rfe_5.values, i) for i in range(x_train_rfe_5.shape[1])], index = x_train_rfe_5.columns) #se usa el xtrain sin la constante
series_6 = series_6.sort_values(ascending = False)
series_6

In [ ]:
##Send file for testing so far
data_5 = all_data[x_train_rfe_5.columns]
%store data_5

In [ ]:
#eliminamos HalfBath  
x_train_rfe_6 = x_train_rfe_5.drop(columns = ["GrLivArea","BsmtSFPoints", "BsmtSFMultPoints","TotalBsmtSF_x_Bsm","TotRmsAbvGrd"], axis = 1)

#Add constant and fit the model
x_train_rfec_6 = sm.add_constant(x_train_rfe_6)
lm_rfe_6 = sm.OLS(y_train, x_train_rfec_6).fit()

In [ ]:
#VIF 6

series_7 = pd.Series([variance_inflation_factor(x_train_rfe_6.values, i) for i in range(x_train_rfe_6.shape[1])], index = x_train_rfe_6.columns) #se usa el xtrain sin la constante
series_7 = series_7.sort_values(ascending = False)
series_7

In [ ]:
##Testing dataset so far
data_5 = all_data[x_train_rfe_6.columns]
%store data_5

In [ ]:
#eliminamos BsmtHalfBath  
x_train_rfe_7 = x_train_rfe_6.drop(columns = ["HouseStyle_1Story", "KitchenQual","MasVnrType_None","TotBathrooms","LotShape", "PavedDrive", "KitchenAbvGr","OverallCond"] , axis = 1)

#Add constant and fit the model
x_train_rfec_7 = sm.add_constant(x_train_rfe_7)
lm_rfe_7 = sm.OLS(y_train, x_train_rfec_7).fit()

In [ ]:
#VIF 7

series_8 = pd.Series([variance_inflation_factor(x_train_rfe_7.values, i) for i in range(x_train_rfe_7.shape[1])], index = x_train_rfe_7.columns) #se usa el xtrain sin la constante
series_8 = series_8.sort_values(ascending = False)
series_8

In [ ]:
#eliminamos SaleCondition_Partial 
x_train_rfe_8 = x_train_rfe_7.drop(columns = ["Street", "Condition2_Norm"], axis = 1)

#Add constant and fit the model
x_train_rfec_8 = sm.add_constant(x_train_rfe_8)
lm_rfe_8 = sm.OLS(y_train, x_train_rfec_8).fit()

In [ ]:
#VIF 8

series_9 = pd.Series([variance_inflation_factor(x_train_rfe_8.values, i) for i in range(x_train_rfe_8.shape[1])], index = x_train_rfe_8.columns) #se usa el xtrain sin la constante
series_9 = series_9.sort_values(ascending = False)
series_9

In [ ]:
#eliminamos 2ndFlrSF   
x_train_rfe_9 = x_train_rfe_8.drop(columns = ["ExterCond", "GarageQual"], axis = 1)

#Add constant and fit the model
x_train_rfec_9 = sm.add_constant(x_train_rfe_9)
lm_rfe_9 = sm.OLS(y_train, x_train_rfec_9).fit()

In [ ]:
#VIF 9

series_10 = pd.Series([variance_inflation_factor(x_train_rfe_9.values, i) for i in range(x_train_rfe_9.shape[1])], index = x_train_rfe_9.columns) #se usa el xtrain sin la constante
series_10 = series_10.sort_values(ascending = False)
series_10

In [ ]:
#eliminamos 1stFlrSF   
x_train_rfe_10 = x_train_rfe_9.drop(columns = ["FullBath", "Functional"], axis = 1)

#Add constant and fit the model
x_train_rfec_10 = sm.add_constant(x_train_rfe_10)
lm_rfe_10 = sm.OLS(y_train, x_train_rfec_10).fit()

In [ ]:
#VIF 10

series_11 = pd.Series([variance_inflation_factor(x_train_rfe_10.values, i) for i in range(x_train_rfe_10.shape[1])], index = x_train_rfe_10.columns) #se usa el xtrain sin la constante
series_11 = series_11.sort_values(ascending = False)
series_11

In [ ]:

cols_construct = [ "MasVnrArea" ,  "OpenPorchSF","3SsnPorch" , "ScreenPorch" , "EnclosedPorch"] 
                    

In [ ]:
#eliminamos New_FlSF  
x_train_rfe_11 = x_train_rfe_10.drop(columns = cols_construct, axis = 1)

#Add constant and fit the model
x_train_rfec_11 = sm.add_constant(x_train_rfe_11)
lm_rfe_11 = sm.OLS(y_train, x_train_rfec_11).fit()

In [ ]:
#VIF 11

series_12 = pd.Series([variance_inflation_factor(x_train_rfe_11.values, i) for i in range(x_train_rfe_11.shape[1])], index = x_train_rfe_11.columns) #se usa el xtrain sin la constante
series_12 = series_12.sort_values(ascending = False)
series_12

In [ ]:
##
#eliminamos Todos los inf 
x_train_rfe_12 = x_train_rfe_11.drop(columns = ["BsmtCond", "LandSlope", "MSZoning_RL", "LotShape"], axis = 1)

#Add constant and fit the model
x_train_rfec_12 = sm.add_constant(x_train_rfe_12)
lm_rfe_12 = sm.OLS(y_train, x_train_rfec_12).fit()

In [ ]:
#VIF 12

series_13 = pd.Series([variance_inflation_factor(x_train_rfe_12.values, i) for i in range(x_train_rfe_12.shape[1])], index = x_train_rfe_12.columns) #se usa el xtrain sin la constante
series_13 = series_13.sort_values(ascending = False)
series_13

In [ ]:
vif_col = list(x_train_rfec_12.columns)
vif_col.remove("const")


In [ ]:
##Envío este dataset para probarlo en el módulo de models
all_data_2 = all_data[vif_col]
%store all_data_2